<a href="https://colab.research.google.com/github/Khushboo-30gothwal/python/blob/main/tutorial_006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# learning methods on graphs

applying all tut_001 to tut_005 in itand make first graph neural networ!

In [47]:
!pip install torch_geometric

first load cora dataset

In [48]:
import torch
from torch_geometric.datasets import Planetoid

In [49]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [50]:
print(dataset)

Cora()


implement a two layer GCN

In [51]:
import torch
import torch.nn.functional as F
from  torch_geometric.nn import GCNConv

In [52]:
class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16,dataset.num_classes)

  def forward(self,data):
    x, edge_index = data.x, data.edge_index
    x = self.conv1(x,edge_index)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x=self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
model = GCN().to(device)

In [55]:
data = dataset[0].to(device)

In [56]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [60]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

evaluate our model on test nodes

In [61]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc: .4f}')

Accuracy:  0.8040
